# Accelerated Database Recovery


## Step 1: Setup the database
Create a database and put a bunch of rows in a table

In [1]:
-- Step 1: Setup the database
USE master
GO
DROP DATABASE IF EXISTS gocowboys
GO
CREATE DATABASE gocowboys
ON PRIMARY
(NAME = N'gocowboys_primary', FILENAME = 'c:\data\gocowboys.mdf', SIZE = 10Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
LOG ON 
(NAME = N'gocowboys_Log', FILENAME = 'c:\data\gocowboys_log.ldf', SIZE = 10Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
GO
ALTER DATABASE gocowboys SET RECOVERY SIMPLE
GO
USE gocowboys
GO
DROP TABLE IF EXISTS howboutthemcowboys
GO
CREATE TABLE howboutthemcowboys (playerid int primary key clustered, playername char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
DECLARE @x int
SET @x = 0
WHILE (@x < 100000)
BEGIN
	INSERT INTO howboutthemcowboys VALUES (@x, 'Jason Whitten returns in 2019')
	SET @x = @x + 1
END
COMMIT TRAN
GO
SET NOCOUNT OFF
GO
USE master
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:19.018

## Step 2: Delete all the rows in the table
Delete all the rows in the table in a user defined transaction

In [2]:
-- Step 2: Delete all the rows in the table in a user defined transaction
-- Make sure ADR is OFF
USE master
GO
ALTER DATABASE gocowboys SET ACCELERATED_DATABASE_RECOVERY = OFF
GO
-- Try to delete a bunch of rows
USE gocowboys
GO
BEGIN TRAN
DELETE from howboutthemcowboys
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(100000 rows affected)

Total execution time: 00:00:05.022

## Step 3: Check how much transaction log space is used

In [3]:
-- Step 3: How much transaction log space is used
-- What is the log space usage
SELECT * FROM sys.dm_db_log_space_usage
GO

(1 row affected)

Total execution time: 00:00:00.149

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
6,10737410048,4223913984,39.33829,1097400320


## Step 4: Does a checkpoint truncate the transaction log?
Compare the used_log_space_in_percent to the value from the previous step

In [4]:
-- Step 4: Does checkpoint truncate the log?
CHECKPOINT
GO
SELECT * FROM sys.dm_db_log_space_usage
go

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.083

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
6,10737410048,4224032768,39.3394,1000140800


## Step 5: How long does it take to rollback all the deletes
Try to rollback the transation and measure the time it takes

In [21]:
-- Step 5: Try to roll it back and measure the time
ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:15.594

## Step 6: What is the log space usage before and after a CHECKPOINT
Check transaction log space usage before and after a checkpoint. This shows how an active transaction holds up log truncation.

In [22]:
-- Step 6: What is the log space usage
SELECT * FROM sys.dm_db_log_space_usage
GO

-- Does checkpoint truncate the log?
CHECKPOINT
GO
SELECT * FROM sys.dm_db_log_space_usage
go

(1 row affected)

Total execution time: 00:00:00.0163236

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
10,10737410048,4401008640,40.98762,1876959232


Commands completed successfully.

Total execution time: 00:00:00.2831751

(1 row affected)

Total execution time: 00:00:00.0112435

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
10,10737410048,66273280,0.6172185,94208


## Step 7: Turn on Accelerated Database Recovery

In [23]:
-- Step 7: Enable Accelerated Database Recovery
USE master
GO
ALTER DATABASE gocowboys SET ACCELERATED_DATABASE_RECOVERY = ON
GO

Commands completed successfully.

Total execution time: 00:00:00.0014138

Commands completed successfully.

Total execution time: 00:00:00.0192079

## Step 8: Try the delete again and check log space usage

In [24]:
-- Step 8: Try deleting all the rows in the table and look at log space usage before and after a CHECKPOINT
-- Try to delete a bunch of rows in a transaction
USE gocowboys
GO
BEGIN TRAN
DELETE from howboutthemcowboys
GO

-- What is the log space usage
SELECT * FROM sys.dm_db_log_space_usage
GO
CHECKPOINT
GO
SELECT * FROM sys.dm_db_log_space_usage
GO

Commands completed successfully.

Total execution time: 00:00:00.0013823

(100000 rows affected)

Total execution time: 00:00:02.1115536

(1 row affected)

...

Total execution time: 00:00:00.0824323

(1 row affected)

Total execution time: 00:00:00.0131580

database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
10,10737410048,1057198080,9.845932,768376832


database_id,total_log_size_in_bytes,used_log_space_in_bytes,used_log_space_in_percent,log_space_in_bytes_since_last_backup
10,10737410048,386236416,3.59711,90112


## Step 9: How fast is a rollback?
See if a rollback is any faster with ADR

In [25]:
-- Step 9: How fast is a rollback?
ROLLBACK TRAN
GO

Commands completed successfully.

Total execution time: 00:00:00.0014140